![image](https://drive.google.com/uc?export=view&id=113XAsG1B7eHjOf2Cn6JKZarzuaa_1rSN)


# Introduction

In [ ]:
# Please, run this while I speak. It requires some minutes.
!pip3 install Pyfhel==2.3.1
!pip3 install tenseal

## The problem

The current scenario of "machine-learning as-a-service" (MLaaS): 
![image](https://drive.google.com/uc?export=view&id=1oDr0yw8i7FfjABqrWLVYiGcCYuIBOpjq)

This scenario introduces **privacy problems**... 
- Privacy of users data: ethical aspects
- Privacy of companies data: secrecy requirements


And leads to the main question we want to address today:
> How can we design software services and mobile apps providing intelligent functionalities (through machine and deep learning solutions) 
while guaranteeing the privacy of user data?

## Privacy-preserving computation in machine and deep learning: tools
There are two main families of tools we can use to preserve the privacy of users' data during the processing of ML and DL models:
1. the first is based on data anonymization and perturbation: it includes *Group-based anonymity" and "Differential privacy"*;
2. the second is based on the (joint) computation of encrypted data: *Multi-party computation* and *Homomorphic encryption".

![image](https://drive.google.com/uc?export=view&id=1POmzkmAYQSZP44tnS6cm13X-aILgxK23)

|                         | Ability to process encrypted data | Processing without the need of  multiple rounds of communication |
|:-----------------------:|:---------------------------------:|:----------------------------------------------------------------:|
|  Homomorphic Encryption |                Yes                |                                Yes                               |
| Multi-party Computation |                Yes                |                                No                                |
|  Group-based Anonymity  |                 No                |                                Yes                               |
|   Differential Privacy  |                 No                |                                Yes                               |

## Homomorphic encryption
Homomorphic encryption (HE) is a family of encryption schemes which allows one to process directly encrypted data, without the need to decrypt them.

![image](https://drive.google.com/uc?export=view&id=1xEhSqQKpWwn8reXm8u1MzqqYn-sn9n7W)

In other words, an encryption function $E$ and its corresponding decryption function $D$ are homomorphic with respect to a class of functions $F$ if, for any function $f \in F$, it is possible to construct a corresponding function $g_f$ such that $f(x) = D(g_f(E(x)))$ for a set of input $x$.

### HE schemes
There are different HE schemes, all based on the same mathematical aspects, but with different peculiarities and uses:

| Category             | Computation                                                       | Example    | Comment                                          |
|----------------------|-------------------------------------------------------------------|------------|--------------------------------------------------|
| Partially HE schemes | Only one class of operations (+ or -)                             | RSA        |                                                  |
| Somewhat HE schemes  | Unbounded number of additions and one single multiplication       | BGN        |                                                  |
| Leveled HE schemes   | A pre-determined number of additions and multiplications          | BFV / CKKS | Current solutions in the HE-DL scenario          |
| Fully HE schemes     | Unbounded number of operations, often binary ones (AND, NOT, ...) | TFHE       | Future (still difficult to configure and manage) |

In the next practical example we will see how the BFV scheme works.

# Demo 1: BFV scheme from scratch

In [ ]:
# Some helper functions.

import numpy as np
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polypow
from math import floor

# Mean and standard deviation of a Gaussian distribution that will be used in the followings.
# Sigma is a "suggested" value for safe BFV, see https://eprint.iacr.org/2019/939.pdf
mu, sigma = 0, 3.2

def Poly(coeffs):
    """
    Helper function to build polynomials, passing a dictionary of coefficients.
    For example, passing {0: 1, 1: -2, 2: 2} returns the polynomial
    2X**2 - 2X + 1
    """
    max_power = max(coeffs.keys())
    _coeffs = np.zeros(max_power + 1)
     
    for i, c in coeffs.items():
        _coeffs[i] = c
        
    return Polynomial(_coeffs)

def pr(p):
    """ 
    Helper function to pretty-print the polynomials, with human order
    of powers, removed trailing .0, etc.
    """
    coefs = p.coef
    res = ""
    
    powers = range(len(coefs)-1, -1, -1)
    for power, coeff in zip(powers, reversed(coefs)):
        if coeff == 0:
            continue
        
        if int(coeff) == coeff:
            coeff = int(coeff)
                  
        sign = "- " if coeff < 0 else "+ "
        
        if power == 0:
            value = abs(coeff)
        elif abs(coeff) != 1:
            value = abs(coeff)
        else:
            value = ""

        power_sign = {0: "", 1: "X"}
        def_power_sign = f"X**{power}"
        
        res += f" {sign}{value}{power_sign.get(power, def_power_sign)}"
        
    if res[1] == "+":
        return res[3:]
    if res[1] == "-":
        return res[1:]

def mod_on_coefficients(polynomial, modulo):
    """
    Apply the modulo on the coefficients of a polynomial.
    """
    coefs = polynomial.coef
    mod_coefs = []
    for c in coefs:
        mod_coefs.append(c % modulo)
        
    return Polynomial(mod_coefs)

def round_on_nearest_integer(polynomial):
    """
    Round the coefficients of a polynomial to the
    nearest integer.
    """
    coefs = polynomial.coef
    round_coefs = []
    for c in coefs:
        round_coefs.append(round(c))
    
    return Polynomial(round_coefs)

The Brakerski-Fan-Vercauteren (BFV) scheme is an homomorphic encryption (HE) scheme which allows the computation of some functions directly on *encrypted data*. 

In particular, it allows the computation of additions and multiplication:

- between ciphertexts and ciphertexts;
- between ciphertexts and plaintexts.

The result of the operations, once decrypted, is the same as if it were applied on the corresponding plaintexts.

The schemes like BFV or also CKKS are based on a hard computation problem called *Ring Learning With Errors*.

Let's understand this problem starting from the bottom.

## Data representation
The data in these schemes is represented by *polynomials*. This holds both when data is encrypted (ciphertexts) and when it is unencrypted (plaintexts).

So, the starting point are common polynomials like:

In [ ]:
p = Poly( ??? )
print(pr(p))

However, there is more. 

### Coefficients
First of all, the coefficients of the polynomials are whole numbers, and are the *remainder* relative to some other number $k$ (that is $\bmod k$).

Take for example $k=12$; we can see the ring as a common clock, in which each our corresponds to an element in the ring.
Adding $3$ to $11$ will result in $2$ instead of $14$.

![modular](https://drive.google.com/uc?export=view&id=1tW_szr-j-7ov2Vw75pJApEzG2qf0-0hO)

It also important to note that in this ring we could make the numbers go from $-5$ to $6$: this is arbitrary, because it is easy to see that a remainder of $-1$ is the same as a remainder of $11$. In the following this consideration will be assumed.

### Polynomial modulus
The second aspect is that also the polynomials themselves are remainder: we define a special polynomial called *polynomial modulus*. Each polynomial used in the scheme is divided by this polynomial modulus and the remainder is taken.

In the HE schemes it is common to take this polynomial modulus in the form $x^n + 1$ where $n$ is a power of $2$. For example, if we take $n=8$ we have the polynomial $x^{8} + 1$.

Put simply, the polynomials employed in the scheme have the following two characteristics:

  1. Their coefficients are modulo $k$, creating a ring from $0$ to $k$;
  2. The maximum degree is $n-1$;

So, all the polynomials we are considering are of the form (assuming $n=8$):
$$ a_7 x^7+a_6 x^6+a_5 x^5+a_4 x^4+a_3 x^3+a_2 x^2+a_1 x+a_0 $$

remembering that each of the 8 coefficients $a_i$ can range from $0$ to $k-1$.

We can also give a powerful graphical representation of this:

![torus](https://drive.google.com/uc?export=view&id=1rM24TY_qPcyJT0InhMcNW_jyYdu0w3f3)

Each loop contains the 12 possible values of the coefficient of one of the powers of $x$ that appears in the polynomial. 

Formally: the polynomials used in the scheme are in the space $R_k = \mathbb{Z}_k[x] / (x^n + 1)$.

### Practical examples
Let's see this in action with a multiplication between polynomials.
We let:
- $n = 8$ (hence, the polynomial modulus is $x^8 + 1$);
- $k = 7$

If we multiply:
- a = $3 x^{4}$ and
- b = $4x^5$

we obtain:
- $12 x^{9}$

After each operation, we have to perform the division with the polynomial modulus and keep the remainder.

$$ 12 x^9 \bmod x^8 + 1 = -12x $$

Also, the result's coefficients have to be taken $\bmod k$.

$$ [-12x]_k = 2x $$

In [ ]:
a = Poly({ ??? })
b = Poly({ ??? })
polynomial_modulus = Poly({ ??? })

prod = a * b

quo, rem = ??? (prod, polynomial_modulus)
final_result = ??? (rem, 7)

print(f"A: {pr(a)}")
print(f"B: {pr(b)}")
print(f"Product of A and B: {pr(prod)}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")
print(f"----------------------")
print(f"Remainder of (A*B) / polynomial modulus: {pr(rem)}")
print(f"Apply also mod k: {pr(final_result)}")

The polynomials used as modulo, $x^n + 1$ are in the family of *cyclotomic polynomials*.

## Encryption using polynomials ring
Now we can go to the encryption/decryption phase.

### Plaintexts, ciphertexts, and keys

A plaintext is a polynomial :
- from the ring with polynomial modulus $\Phi_n = x^n + 1$, with $n$ being a power of $2$;
- coefficients modulus $p$ (which will be denoted as *plaintext coefficient modulus*). 

A ciphertext is composed of (at least) **two polynomials**:
- from the ring with the same polynomial modulus;
- but with the coefficients modulus $q$ (which will be denoted as *ciphertext coefficient modulus*) which is typically much larger than $p$.

In real cases of BFV applications, it is common to take values of $n$ of at least $2048$ while for $q$, usually very large values are used.

To make a simpler example, we will use much smaller values, for example $n=16$, $p=7$ and $q=874$. Note that these parameter are not *secure* for real cases (and we will see why...).

In [ ]:
n = ??
polynomial_modulus = Poly({ ??? })
p = ?
q = ???

print(f"Plaintext coefficient modulo: {p}")
print(f"Ciphertext coefficient modulo: {q}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")

Let's generate the private key, $k_s$: the secret key corresponds to a random polynomial of maximum degree $n$ with coefficients of either $-1, 0, 1$. 

For example:

In [ ]:
coeffs = {}
for i in range(0, n):
    coeffs[i] = ???
    
s = Poly(coeffs)
print(f"Secret key: {pr(s)}")

Next we generate a public key.

For this we need:

  1. a random polynomial from the ciphertext space, hence with coefficients modulo $q$, which we'll call $a$;
  2. an *error polynomial*, which is small in the sense that has small coefficients drawn from a discrete Gaussian distribution. We will use this once, then it is discarded. We'll call this $e$.

At this point we can define the public key as the pair of polynomials:

$$k_p = ([{-(a k_s + e)}]_{\Phi_n, q} , a)$$

where the polynomial arithmetic is done modulo the polynomial modulus and the coefficient arithmetic modulo $q$.

In [ ]:
coeffs = {}
for i in range(0, n):
    coeffs[i] = np.random.randint(0, q)

a = Poly(coeffs)

dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, n):
    coeffs[ind] = round(dist[ind])


e = Poly(coeffs)

pk_0 = ???
pk_0 = divmod(pk_0, polynomial_modulus)[1]
pk_0 = mod_on_coefficients(pk_0, q)

pk_1 = ???

pk = ???

print(f"a: {pr(a)}\n")
print(f"e: {pr(e)}\n")
print(f"Public key: {pr(pk_0)}, {pr(pk_1)}\n")

It is interesting to note that, in a sense, the secret key is contained in the public key. However, to mask it, it is multiplied with the polynomial $a$ which has coefficients in the range $\bmod q$ (it is quite "big"). 

However, also $a$ is contained in the public key; a little error $e$ is added to $a*s$ and this makes it computationally hard to retrieve $s$. 
This is the exact definition of the *Ring Learning with Errors*.

To recap:

$$ s \rightarrow \text{"small" polynomial (coefficients in -1, 0, 1)}$$
$$ a \rightarrow \text{"big polynomial (coefficients in mod q)}$$
$$ e \rightarrow \text{"small noise" polynomial (coefficients drawn by a Gaussian distribution)}$$

$$ k_s$$
$$ k_p = \left ( \left [ -(a k_s + e) \right ]_{\Phi_n, q} , a \right ) $$

### Encryption
Encryption transforms a plaintext - encoded in the form of a polynomial with coefficients modulo $p$ (that, is in the space $R_p = \mathbb{Z}_p[{x}]/(x^n + 1)$) into a ciphertext.
A fresh ciphertext, in BFV, is a pair of polynomials with coefficients modulo $q$ (that is, they are in the space $R_{\Phi_n, q} = \mathbb{Z}_{\Phi_n, q}[{x}]/(x^n + 1)$). 

In [ ]:
m = Poly({0: 1, 1: 1, 2: 1})
print(f"Plain message: {pr(m)}")

To perform the encryption we need three "small" polynomials:

1. Two error polynomials ("small error" polynomials), extracted from a discrete Gaussian distribution (similarly to the one used in the public key);
2. A "small" polynomial, $u$, which has coefficients drawn from (-1, 0, 1), similar to $s$.

In [ ]:
dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, 16):
    coeffs[ind] = round(dist[ind])

e_1 = Poly(coeffs)

dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, 16):
    coeffs[ind] = round(dist[ind])

e_2 = Poly(coeffs)

coeffs = {}
for i in range(0, 16):
    coeffs[i] = np.random.randint(-1, 2)
    
u = Poly(coeffs)

print(f"e_1: {pr(e_1)}\n")
print(f"e_2: {pr(e_2)}\n")
print(f"u: {pr(u)}\n")

We use these polynomials in the encryption phase, then we discard them.
The ciphertext is represented by two polynomials calculated as:
$$ct = \left ( \left [ k_{p_0} u + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m \right ]_{\Phi_n, q} , \left [k_{p_1} u + e_2 \right ]_{\Phi_n, q} \right)$$

Which can be rewritten, expanding $k_p$:
$$ct = \left ( \left [ -asu -eu + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m \right ]_{\Phi_n, q} , \left [ au + e_2 \right ]_{\Phi_n, q} \right)$$

The initial message had coefficients in the range of $\bmod p$. However, they have been *scaled* by $\left \lfloor \frac{q}{p} \right \rfloor$. This means that, in the ciphertext, the message has coefficients in the range of $\bmod q$. 
Morover the role of $u$ become clear: $u$ is a mask which changes at every encryption, and is responsible for the property of confusion typical of HE schemes. The same plaintext message will always have a different encryption.

The other terms in the ciphertext *hide* the scaled message, including $-asu$ (which is the biggest one, in terms of coefficients).
On the other hand, having $-asu$ and $au$ in the ciphertext is the key for the decryption phase.

In [ ]:
ct0 = pk[0] * u + e_1 + floor((q/p)) * m
ct0 = divmod(ct0, polynomial_modulus)[1]
ct0 = mod_on_coefficients(ct0, q)

ct1 = pk[1] * u + e_2
ct1 = divmod(ct1, polynomial_modulus)[1]
ct1 = mod_on_coefficients(ct1, q)

print(f"Ciphertext: {pr(ct0)}, {pr(ct1)}")

### Decryption
The decryption is relatively simple.

In fact, you may have noted that the biggest error term, which hides our message in the ciphertext, is given by $-asu$.
However, if we are the entity who crypted the message:

  1. we have $s$;
  2. we have $au$, because it is contained in the second term of the ciphertext.

So, we just have to multiply the second term of the ciphertext with $s$, and sum it with the first term of the ciphertext:

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -asu -eu + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m + asu + se_2 \right ]_{\Phi_n, q} $$

We end up with:

$$ \left [ \left \lfloor \frac{q}{p} \right \rfloor m -eu + e_1 + se_2 \right]_{\Phi_n, q} $$

Inside this polynomial we have the scaled message summed to some noise. If the noise is not too big, we can recover the message.

To do that, we just try to make the modulo with the polynomial modulus, than to apply $\bmod q$ to the coefficients of the resulting polynomial.

In [ ]:
decrypt = ???
decrypt = divmod(decrypt, polynomial_modulus)[1]
decrypt = mod_on_coefficients(decrypt, q)

print(f"Scaled plaintext + errors: {pr(decrypt)}")

If we rescale...

In [ ]:
decrypt = ???
print(f"De-scaled plaintext plus errors: {pr(decrypt)}")

Don't forget to round to the nearest integer and make the modulo $\bmod p$!

In [ ]:
decrypt = round_on_nearest_integer(decrypt)
decrypt = mod_on_coefficients(decrypt, p)

print(f"Plain starting message: {pr(m)}")
print(f"Final decryption result: {pr(decrypt)}")

This is exactly our message!

If the coefficients are too noisy, they will be rounded to another integer, and we will end up with wrong terms. The amount of room for noise can be adjusted by making the ratio $q/p$ larger or smaller.

We are ready to define some other helper functions to automatize the process of encryption and decryption.

In [ ]:
def generate_keys(n, q):
    """
    Generate public and secret key, using the parameters
    d, q passed as parameters.
    
    Returns sk, pk.
    """
    polynomial_modulus = Poly({0: 1, n: 1})
    coeffs = {}
    
    # Secret key
    for i in range(0, n):
        coeffs[i] = np.random.randint(-1, 2)

    sk = Poly(coeffs)
    
    # Public key
    coeffs = {}
    for i in range(0, n):
        coeffs[i] = np.random.randint(0, q)

    a = Poly(coeffs)

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e = Poly(coeffs)

    pk_0 = -((a * sk) + e)
    pk_0 = divmod(pk_0, polynomial_modulus)[1]
    pk_0 = mod_on_coefficients(pk_0, q)

    pk_1 = a

    pk = (pk_0, pk_1)
    
    return sk, pk

def encrypt_poly(m, pk, n, p, q):
    """
    Encrypt a polynomial message given the public key.
    Returns the encrypted polynomial.
    """
    polynomial_modulus = Poly({0: 1, n: 1})

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e_1 = Poly(coeffs)

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e_2 = Poly(coeffs)

    coeffs = {}
    for i in range(0, n):
        coeffs[i] = np.random.randint(-1, 2)

    u = Poly(coeffs)

    ct0 = pk[0] * u + e_1 + floor((q/p)) * m
    ct0 = divmod(ct0, polynomial_modulus)[1]
    ct0 = mod_on_coefficients(ct0, q)
    
    ct1 = pk[1] * u + e_2 
    ct1 = divmod(ct1, polynomial_modulus)[1]
    ct1 = mod_on_coefficients(ct1, q)
    
    return (ct0, ct1)

def decrypt_poly(ct, ks, n, p, q):
    """
    Decrypts an encrypted polynomial, given the secret key.
    Returns the decrypted polynomial.
    """
    polynomial_modulus = Poly({0: 1, n: 1})
    decrypt = Poly({0: 0})
    
    for i, term in enumerate(ct):
        decrypt += ct[i] * polypow(ks.coef, i, 1000)
    
    decrypt = divmod(decrypt, polynomial_modulus)[1]
    decrypt = mod_on_coefficients(decrypt, q)
    
    decrypt = decrypt * p/q
    decrypt = round_on_nearest_integer(decrypt)
    decrypt = mod_on_coefficients(decrypt, p)
    
    return decrypt    

## Homomorphic operations
Obviously, we are interested in the special aspect of these HE schemes: the possibility to compute homomorphic additions and  multiplications.

This means that we can add and multiply numbers while they are still encrypted, without having to decrypt them.

### Homomorphic addition (between ciphertexts)
The simplest case is the addition of two ciphertexts. 
Starting from $ct_1$ and $ct_2$, which are the encryption of messages $m_1$ and $m_2$ respectively, we can compute $ct_3 = ct_1 + ct_2$.
The decryption of $ct_3$ is equal to $m_1 + m_2$.

Obviously, this assuming the correctness of the processes and suitable encryption parameters.

In [ ]:
n = 16
p = 7
q = 874

ks, kp = ??? (n, q)

m1 = Poly({0: 1, 1: 1, 2: 1})
m2 = Poly({1: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected sum: {pr(m1+m2)}")
print(f"-------------------------")

enc_m1 = ??? (m1, kp, n, p, q)
enc_m2 = ??? (m2, kp, n, p, q)
print(f"First ciphertext: {pr(enc_m1[0])}, {pr(enc_m1[1])}\n")
print(f"Second ciphertext: {pr(enc_m2[0])}, {pr(enc_m2[1])}\n")

To perform the addition between the two ciphertexts, it is sufficient to add them element-wise.
In the followings $u_1$ and $u_2$, as well as $e_1, \ldots, e_4$ denote the "small" and "small errors" polynomials created during the encryption phase.

$$ enc\_m_1 + enc\_m_2 = \left ( \left [k_{p_0}u_1 + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m_1 \right ]_{\Phi_n, q} , \left [k_{p_1}u_1 + e_2 \right]_{\Phi_n, q} \right ) + \left ( \left [k_{p_0}u_2 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_2 \right ]_{\Phi_n, q} , \left [ k_{p_1}u_1 + e_4 \right]_{\Phi_n, q} \right ) $$

$$ enc\_m_1 + enc\_m_2 = \left ( \left [k_{p_0}(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor (m_1 + m_2) \right ]_{\Phi_n, q} , \left [ k_{p_1}(u_1 + u_2) + e_2 + e_4 \right ]_{\Phi_n, q} \right ) $$

We are interested in the term $\left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2)$, which is the scaled sum of our messages; however, new noise terms appeared in the ciphertext.

Let's try to decrypt this, multiplying the second term of the ciphertext with $s$ and summing it up. Also, expand $k_p$:

<!-- $$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ { -as(u_1 + u_2) - e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + (a(u_1 + u_2) + e_2 + e_4)s } \right ]_{\Phi_n, q} $$

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -as(u_1 + u_2) - e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + as(u_1 + u_2) + s(e_2 + e_4) \right ]_{\Phi_n, q} $$ -->

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + s(e_2 + e_4) \right ]_{\Phi_n, q} $$

- We have some new noise terms. 
- the noise polynomials become a problem if one of their coefficients is larger than $\frac{q}{2p}$, which, in our case, is around $62$.

The new noise terms are:

  1. $-e(u_1 + u_2)$: this is the product of a "small noise" polynomial with the sum of two "small" polynomials (with coefficients $-1, 0, 1$). This can be quite big, because sometimes the sum of $-1, 0, 1$ will be close to $0$, while other times it will become $2, 3, 4...$. This can reach $62$ pretty fast.
  2. $e_1 + e_3$: this is only the sum of two "small noise" polynomials. It's less worrying than the first.
  3. $s(e_2 + e_4)$: this is bad similarly to the the first.

Can we graphically visualize how bad is this?

![image](https://drive.google.com/uc?export=view&id=1ybxLOWVvz4T-0o1BNxYDHaDzqVFXzrqw)

- After just one addition, there is a decent probability that a coefficient of the noise polynomials will be higher than $62$, leading to an error in the decryption phase.

To have more room, we have to increment the ratio $q/p$.

Let's compute an addition.

In [ ]:
def add_ciphertexts(enc_m1, enc_m2, n, q):
    polynomial_modulus = Poly({0: 1, n: 1})
    enc_sum_0 = ???
    enc_sum_1 = ???

    enc_sum_0 = divmod(enc_sum_0, polynomial_modulus)[1]
    enc_sum_0 = mod_on_coefficients(enc_sum_0, q)

    enc_sum_1 = divmod(enc_sum_1, polynomial_modulus)[1]
    enc_sum_1 = mod_on_coefficients(enc_sum_1, q)

    enc_sum = (enc_sum_0, enc_sum_1)
    return enc_sum

enc_sum = add_ciphertexts(enc_m1, enc_m2, n, q)
print(f"Encrypted sum: {pr(enc_sum[0])}, {pr(enc_sum[1])}\n")

decrypted = decrypt_poly(enc_sum, ks, n, p, q)
print(f"Expected sum: {pr(m1+m2)}")
print(f"Decrypted result: {pr(decrypted)}")

Remember that we are in a ring, so $+6 \equiv -1$, etc.
If the result is not correct, try to encrypt the messages again and re-computing the addition.

### Homomorphic multiplication (between ciphertexts)
The multiplication between ciphertexts is more complicated than the addition and can cause an important increment of the noise in the resulting ciphertext.

Key points:
1. At some point, we will have a term: $\left \lfloor \frac{q}{p} \right \rfloor m_1 * \left \lfloor \frac{q}{p} \right \rfloor m_2$ which is $\frac{q^2}{p^2}m_1m_2$ -> we will multiply the ciphertexts also with $\frac{p}{q}$ in order to come back to $\left \lfloor \frac{q}{p} \right \rfloor m_1m_2$;
2. The multiplication between two ciphertexts produces a three-terms ciphertext;
3. If we use the encryption parameters that we used in the addition case ($t=7$, $q=874$) we have a problem if a coefficient in the result polynomial will be higher than $\left \lfloor \frac{q}{2t} \right \rfloor = 62$.

Let's inspect the distribution of the coefficients in one multiplication, for all the noise terms we obtained:

![image](https://drive.google.com/uc?export=view&id=10Z7i9SL7QTmjWDxn4gZA08c_a_WI-ZnV)

It is basically sure that this will not work: in the histogram for $-e * u_1 * m_2$ we see that there is a very high probability of coefficients being in the range $[{-500, 500}]$. We have to increment the ratio $\frac{q}{p}$ to have more room with noise. Let's try with $424242$:

In [ ]:
n = 16
p = 7
q = 424242

ks, kp = generate_keys(n, q)

m1 = Poly({0: 1, 1: 1, 2: 3})
m2 = Poly({0: -1, 1: -2, 2: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected product: {pr(m1*m2)}")
print(f"-------------------------")

enc_m1 = encrypt_poly(m1, kp, n, p, q)
enc_m2 = encrypt_poly(m2, kp, n, p, q)
print(f"First ciphertext: {pr(enc_m1[0])}, {pr(enc_m1[1])}\n")
print(f"Second ciphertext: {pr(enc_m2[0])}, {pr(enc_m2[1])}")

In [ ]:
def multiply_ciphertexts(ct1, ct2, n, p, q):
    polynomial_modulus = Poly({0: 1, n: 1})
    
    c0 = (p/q) * ct1[0] * ct2[0]
    c0 = divmod(c0, polynomial_modulus)[1]
    c0 = mod_on_coefficients(c0, q)
    c0 = round_on_nearest_integer(c0)
    
    c1 = (p/q) * (ct1[0] * ct2[1] + ct1[1] * ct2[0])
    c1 = divmod(c1, polynomial_modulus)[1]
    c1 = mod_on_coefficients(c1, q)
    c1 = round_on_nearest_integer(c1)
    
    c2 = (p/q) * ct1[1] * ct2[1]
    c2 = divmod(c2, polynomial_modulus)[1]
    c2 = mod_on_coefficients(c2, q)
    c2 = round_on_nearest_integer(c2)
    
    return (c0, c1, c2)    

mult = multiply_ciphertexts(enc_m1, enc_m2, n, p, q)
decrypted = decrypt_poly(mult, ks, n, p, q)

print(f"Expected product: {pr(m1*m2)}")
print(f"Decrypted result: {pr(decrypted)}")

### Homomorphic operations between ciphertexts and plaintexts
An important aspect of schemes like BFV is the possibility to compute additions and multiplications not only between ciphertexts, but also between a ciphertext and a plaintext.

This is very important for HE-ML: if a third party wants to run a ML model on encrypted data, it is useless to encrypt the model. Not only, this is actually wasteful because **operations between ciphertexts and plaintexts create much less noise**.

They are quite easy with respect to the ones we saw before.
Let $ct = (ct_0, ct_1, \ldots, ct_n)$ be an encrypted polynomial and $pt$ a plain polynomial.

$$ ct + pt = (ct_0 + \left \lfloor \frac{q}{p} \right \rfloor pt, ct_1, \ldots, ct_n) $$
$$ ct * pt = (ct_0 * pt, ct_1 * pt, \ldots, ct_n * pt) $$

In [ ]:
def add_cipher_and_plain(ct, pt, p, q):
    ct = list(ct)
    ct[0] = ct[0] + floor((q/p))*pt
    ct = tuple(ct)
    
    return ct

def multiply_cipher_and_plain(ct, pt):
    
    new_ct = []
    for elem in ct:
        new_ct.append(elem * pt)
    
    return tuple(new_ct)

In [ ]:
n = 16
p = 7
q = 424242

ks, kp = generate_keys(n, q)

m1 = Poly({0: 1, 1: 1, 2: 3})
m2 = Poly({0: -1, 1: -2, 2: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected sum: {pr(m1+m2)}")
print(f"Expected product: {pr(m1*m2)}")
print(f"-------------------------")

print(f"Encrypting first message...")
enc_m1 = encrypt_poly(m1, kp, n, p, q)

add_result = add_cipher_and_plain(enc_m1, m2, p, q)
mult_result = multiply_cipher_and_plain(enc_m1, m2)

print(f"--------------------------")
print(f"Decrypted sum: {pr(decrypt_poly(add_result, ks, n, p, q))}")
print(f"Decrypted product: {pr(decrypt_poly(mult_result, ks, n, p, q))}")

In the case of ciphertext-plaintext addition, virtually we are not adding any noise to the ciphertext: in fact, if the plain message is $m_2$, we are adding to the ciphertext $\left \lfloor \frac{q}{p} \right \rfloor m_2$ which will be summed to $\left \lfloor \frac{q}{p} \right \rfloor m_1$. However, adding a polynomial may lead a coefficient in the ciphertext to go above $t$ or also $\frac{q}{2t}$.

For multiplications, the problem is a more complicated; however, generally, the noise incremenet depends on the "dimension" of the plaintext message. This is still an operation which produce much less noise than ciphertext-ciphertext multiplications and do not require a relinearization phase after.

# Demo 2: Helper libraries for HE

In this part of the tutorial we will learn how to use an helper Python library for HE, namely [Pyfhel](https://github.com/ibarrond/Pyfhel).
Note: we will use the version 2.x of the library for practical reasons, but check out the new version 3!

## Encryption parameters
To use BFV, we have to choose two encryption parameters:
- $m$: the degree of the cyclotomic polynomial (i.e., the maximum degree of the polynomials);
- $p$: the plaintext coefficients modulo.

The third parameter, $q$, which is the ciphertext coefficients modulo, is chosen automatically by the library in order to be safe (with a 128 bit level of security---stronger levels of security may be chosen if desired).

In [ ]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt

HE = ???
HE.???
HE.???

print(HE)

Even though the BFV scheme was born for integers, Pyfhel 2.x let the user use a *fractional encoder* which makes it possible to use the BFV with fractional numbers, with fixed precision.

In [ ]:
a = 127.15717263
b = -2.128965182
ctxt1 = HE.???
ctxt2 = HE.???

ctxtSum = ctxt1 + ctxt2
ctxtSub = ctxt1 - ctxt2
ctxtMul = ctxt1 * ctxt2

resSum = HE.???(ctxtSum)
resSub = HE.???(ctxtSub) 
resMul = HE.???(ctxtMul)

print(f"Expected sum: {a+b}, decrypted sum: {resSum}")
print(f"Expected sub: {a-b}, decrypted sum: {resSub}")
print(f"Expected mul: {a*b}, decrypted sum: {resMul}")

Noise is added to numbers...

In [ ]:
print(f"Starting Noise Budget: {HE.???(ctxt1)}")

print(f"sum Noise Budget: {HE.???(ctxtSum)}")
print(f"prod Noise Budget: {HE.???(ctxtMul)}")

In [ ]:
ptxt1 = HE.???(-2.128965182)

print(f"ctxt-ptxt prod Noise Budget: {HE.???(ctxt1 * ptxt1)}")

Can we use Pyfhel to process encrypted data, with a DL model? Let's start with a simple linear layer...

$$ y = Wx^T + b $$



Let's try with:
$$
W = \begin{bmatrix} 
1 & 2\\  
1 & 1\\
3 & 1
\end{bmatrix}
$$

$$
b = \begin{bmatrix} 
0 \\
1 \\
2  
\end{bmatrix}
$$

and

$$
x = \begin{bmatrix} 
1 & 2\\  
\end{bmatrix}
$$

we should obtain:
$$
y = (\begin{bmatrix} 
1 & 2\\  
1 & 1\\
3 & 1
\end{bmatrix} * 
\begin{bmatrix} 
1\\  
2
\end{bmatrix}) + 
\begin{bmatrix} 
0 \\ 
1 \\
2   
\end{bmatrix}
=  
\begin{bmatrix} 
5\\
4\\
7
\end{bmatrix}
$$

In [ ]:
import numpy as np
class LinearLayer:
    def __init__(self, weights, bias=None):
        self.weights = weights
        self.bias = bias

    def __call__(self, t):
        
        result = np.array([np.sum(t * row) for row in self.weights])

        if self.bias is not None:
            result = np.array(result + self.bias)

        return result

First, we create weights and biases:

In [ ]:
W = np.array([[1, 2], [1, 1], [3, 1]])

In [ ]:
b = np.array([0, 1, 2])

Then, the input:

In [ ]:
x = np.array([1, 2])

Create a linear layer:

In [ ]:
lin_layer = LinearLayer(W, b)

In [ ]:
result = lin_layer(x)
print(result)

What if our x was encrypted?

In [ ]:
encode  = lambda x: HE.encodeInt(x)
encrypt = lambda x: HE.encryptInt(x)
decrypt = lambda x: HE.decryptInt(x)

In [ ]:
encrypted_x = np.stack(np.vectorize(encrypt)(x), axis=0)

In [ ]:
encoded_W = np.stack(np.vectorize(encode)(W), axis=0)
encoded_b = np.stack(np.vectorize(encode)(b), axis=0)

In [ ]:
encoded_lin_layer = LinearLayer(encoded_W, encoded_b)

In [ ]:
encrypted_result = encoded_lin_layer(encrypted_x)
print(encrypted_result)

In [ ]:
decrypted_result = np.stack(np.vectorize(decrypt)(encrypted_result), axis=0)
print(decrypted_result)

# Designing HE-optimized DL models: a methodology
> How can we design machine and deep learning models able to operate on encrypted data with HE-BFV?

![image](https://drive.google.com/uc?export=view&id=1d8vBQueEPWdoabEkvoG-q-xZE64CGtjT)

## 1. Model approximation
Replacing the processing layers of $F(\cdot)$ that are not compliant with the BFV scheme with those relying only on additions and multiplications.

The output of this step is an approximated model $\varphi(\cdot)$ comprising processing layers that are HE-compliant.

- Maximum pooling -> Average pooling
- Layer normalization -> Batch normalization
- ReLU/tanh -> Square

## 2. Model encoding
- Weights have been encoded according to the BFV scheme and the parameters $\Theta = [n,p,q]$;
- Properly setting the parameters $\Theta$ in a HE processing is still an open research area and generally the selection of the value of these parameters follows a “trial-and-error” approach.

Some rules of thumb:
- $n \geq 4096$;
- $p \geq 2^{16}$;
- $q$ chosen from known safe values (usually $\geq 10^{30}$).

## 3. Model validation
Two main purposes:
- $\varphi_\Theta(\cdot)$ is evaluated to check if the selected configuration $\Theta$ provides a sufficient amount of NB in the processing of the ciphertexts;
- The loss in accuracy of $\varphi_\Theta(\cdot)$ w.r.t. $F(\cdot)$ is evaluated.

To achieve both goals a (possibly large) set of raw messages (i.e., a validation set) is processed by $\varphi_\Theta(\cdot)$.

> If the constraint on the NB is satisfied and the loss in accuracy is below a user-defined threshold (e.g., 1% or 5%): $\varphi_\Theta(\cdot)$ becomes the privacy-preserving version of $F(\cdot)$ to be considered. 

![image](https://drive.google.com/uc?export=view&id=1qfe5oDb7tQU4q-6Zao6Thvruc97AYBHg)



# Demo 3: PyCrCNN: HE-DL made easy

PyCrCNN is a Python library which can automatically process your Torch CNNs on encrypted data, with minimal code.

In [ ]:
!git clone https://github.com/AlexMV12/PyCrCNN
import os, sys
module_path = os.path.abspath(os.path.join('PyCrCNN/'))
sys.path.append(module_path)

In [ ]:
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Convolutional Neural Networks
The simplest model family we can use with HE is Convolutional Neural Networks (CNNs).
They are well suited for two reasons:
- they are mainly composed of polynomial operations (apart from the activation functions, more on this later...);
- the are feed-forward: we can estimate the total number of consecutive multiplications on the inputs.

In [ ]:
transform = transforms.ToTensor()

train_set = torchvision.datasets.???(
    root = './data',
    train=True,
    download=True,
    transform=transform
)

test_set = torchvision.datasets.???(
    root = './data',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=50,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=50,
    shuffle=True
)

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

def train_net(network, epochs, device):
    optimizer = optim.Adam(network.parameters(), lr=0.001)
    for epoch in range(epochs):

        total_loss = 0
        total_correct = 0

        for batch in train_loader: # Get Batch
            images, labels = batch 
            images, labels = images.to(device), labels.to(device)

            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)
        
        print(f"Epoch {epoch+1}, loss: {total_loss:.2f}, accuracy: {total_correct} / {len(train_set)}")

        
def test_net(network, device):
    network.eval()
    total_loss = 0
    total_correct = 0
    
    with torch.no_grad():
        for batch in test_loader: # Get Batch
            images, labels = batch 
            images, labels = images.to(device), labels.to(device)

            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)

        accuracy = round(100. * (total_correct / len(test_loader.dataset)), 4)

    return total_correct / len(test_loader.dataset)

In [ ]:
CNN = nn.Sequential(
            ???
        )

In [ ]:
CNN.to(device)
train_net(CNN, 5, device)
acc = test_net(CNN, device)

In [ ]:
print(f"The CNN model obtained an accuracy of {acc}.")

## Approximating
As we know, there are some operations that cannot be performed homomorphically on encrypted values. Most notably, these operations are division and comparison. It is possible to perform only linear functions.

Consequently, in the CNN we used, we can not use `tanh()`. This is because we cannot apply its non-linearities.


One of the most common approach is to replace it with a simple polynomial function, for example a square layer (which simply performs $x \rightarrow x^2$).

We define the model with all the non-linearities removed **approximated**. This model can be re-trained, and it will be ready to be used on encrypted values.

In [ ]:
class Square(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, t):
        return torch.pow(t, 2)

In [ ]:
CNN_Approx = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            ???
            nn.AvgPool2d(kernel_size=4),

            nn.Flatten(),

            nn.Linear(200, 10),
        )

In [ ]:
CNN_Approx.to(device)
train_net(CNN_Approx, 5, device)
acc = test_net(CNN_Approx, device)

In [ ]:
print(f"The approximated CNN obtained an accuracy of {acc}.")

## Encoding
Let's use PyCrCNN to automate the processing of an encrypted image.
  
Let's remember that, in order to be used with the encrypted values, also the weights of the models will have to be **encoded**. This means that each value in the weights of each layer will be encoded in a polynomial.

We can now use a function to "convert" a PyTorch model to a list of sequential HE-ready-to-be-used layers (`sequential`):

In [ ]:
from pycrcnn.he.HE import BFVPyfhel_Fractional
from pycrcnn.model.sequential import Sequential

## Encrypted processing

Let's list the activities that we will now do:
  1. Create a PyCrCNN BFV HE context, specifiying the encryption parameters `m` (polynomial modulus degree) and `p` (plaintext modulus). Let's remember that `q` will be chosen automatically in order to guarantee a 128-bit RSA equivalent security;
  2. Convert our Torch approximated model to a list of layers able to work on matrices of encrypted values. The weights will be encoded;
  3. Encrypt an image from our testing set;
  4. Verify that the final classification result is correct.

In [ ]:
import matplotlib.pyplot as plt

images, labels = next(iter(test_loader))

sample_image = images[0]
sample_label = labels[0]

In [ ]:
plt.imshow(sample_image[0], cmap='gray', interpolation='none')

In [ ]:
sample_label

We will create two PyCrCNN HE contexts: the one we will use to encrypt the image (`HE_Client`), and the one we will use to process the encrypted image (`HE_Server`). We will need to transfer the public key and the relinearization key in order to allow the server to compute some operations on encrypted data.

In [ ]:
encryption_parameters = {
    "m": ???,
    "p": ???,
}

# Context generation
HE_Client = BFVPyfhel_Fractional(**encryption_parameters)
HE_Client.???
HE_Client.???

public_key = HE_Client.get_public_key()
relin_key  = HE_Client.get_relin_key()

HE_Server = BFVPyfhel_Fractional(**encryption_parameters)
HE_Server.load_public_key(public_key)
HE_Server.load_relin_key(relin_key)

In [ ]:
encoded_model = Sequential(???, CNN_Approx)

In [ ]:
encrypted_image = ???.encrypt_matrix(sample_image.unsqueeze(0).numpy())

Differences

In [ ]:
with torch.no_grad():
    expected_output = ???(sample_image.unsqueeze(0))

In [ ]:
start_time = time.time()
encrypted_output = ???(encrypted_image, debug=False)

requested_time = round(time.time() - start_time, 2)

result = HE_Client.decrypt_matrix(encrypted_output)
difference = expected_output.numpy() - result

print(f"\nThe encrypted processing of one image requested {requested_time} seconds.")
print(f"\nThe expected result was:")
print(expected_output)

print(f"\nThe actual result is: ")
print(result)

print(f"\nThe error is:")
print(difference)    

In this case we were not able to examine the NB evolution during the computation, because in order to compute the NB we need the secret key. If we use the same PyCrCNN HE context both to encrypt and to process the data, then we will also see the evolution of the NB after each layer.

# Contacts
- Manuel Roveri (manuel.roveri@polimi.it)

Manuel Roveri received the Dr. Eng. degree in Computer Science Engineering from the Politecnico di Milano (Italy) in June 2003, the MS in Computer Science from the University of Illinois at Chicago (USA) in December 2003 and the Ph.D. degree in Computer Engineering from the Politecnico di Milano (Italy) in May 2007. He has been Visiting Researcher at Imperial College London (UK) in 2011. Currently, he is an Associate Professor at the Department of Electronics and Information of the Politecnico di Milano (Italy).

Current research activity addresses Embedded and Edge Artificial Intelligence, Tiny Machine and Deep Learning, and Learning in nonstationary/evolving environments.

- Alessandro Falcetta (alessandro.falcetta@polimi.it)

Alessandro Falcetta received the Dr. Eng. degree in Computer Science and Engineering in 2018 and the MS in Computer Science and Engineering in 2020, both from Politecnico di Milano, Italy. Currently, he is a PhD candidate in Information Technology at Dipartimento di Elettronica, Informazione e Bioingegneria (DEIB) at Politecnico di Milano, Italy.


# References
- S. Disabato, A. Falcetta, A. Mongelluzzo, and M. Roveri, “A privacy preserving distributed architecture for deep-learning-as-a-service,” in 2020 International Joint Conference on Neural Networks (IJCNN). IEEE, 2020, pp. 1–8. [GitHub](https://github.com/AlexMV12/PyCrCNN)
- A. Falcetta, and M. Roveri, "Privacy-preserving deep learning with homomorphic encryption: An introduction", Computational Intelligence Magazine (August 2022). [GitHub](https://github.com/AlexMV12/Introduction-to-BFV-HE-ML)
- A.Falcetta, M. Roveri, "Privacy-preserving time series prediction with temporal convolutional neural networks", in 2022 International Joint Conference on Neural Networks (IJCNN), [GitHub](https://github.com/AlexMV12/PINPOINT)
- J. Fan, and F. Vercauteren, "Somewhat Practical Fully Homomorphic Encryption", Cryptology ePrint Archive (2012)
- F. Boemer, Y. Lao, R. Cammarota, and C. Wierzynski, “ngraph-he: a graph compiler for deep learning on homomorphically encrypted data,” in Proceedings of the 16th ACM International Conference on Computing Frontiers, 2019, pp. 3–13.
- Y. LeCun, B. Boser, J. S. Denker, D. Henderson, R. E. Howard, W. Hubbard, and L. D. Jackel, “Backpropagation applied to handwritten zip code recognition,” Neural computation, vol. 1, no. 4, pp. 541–551, 1989. 
- B. C. Stahl and D. Wright, “Ethics and privacy in ai and big data: Implementing responsible research and innovation,” IEEE Security & Privacy, vol. 16, no. 3, pp. 26–33, 2018.
- E. P. Council of European Union, “Regulation (eu) no 2016/679, article 4(1),” 2016. 


# Other resources


Other useful resources on this topic:
- [SoK: Privacy-preserving Deep Learning with Homomorphic Encryption](https://arxiv.org/abs/2112.12855)
- [Microsoft SEAL](https://github.com/microsoft/SEAL)
- [Homomorphic Encryption Standardization](https://homomorphicencryption.org/)

**Don't miss our accepted paper here at IJCNN: "Privacy-preserving time series prediction with temporal convolutional neural networks" we will present on Friday, July 22nd - 4.00PM!**